<img style="float: left; margin: 30px 15px 15px 15px;" src="https://oci02.img.iteso.mx/Identidades-De-Instancia/ITESO/Logos%20ITESO/Logo-ITESO-Principal.jpg" width="500" height="250" /> 
    
    
# <font color='navy'> Backtest

<font color='black'>

- Luis Fernando Márquez Bañuelos

## <font color='cornflowerblue'> Supuestos

- Commision: 0.125%
- No taxes
- Margin Acc: 50%
- Cash: $1,000,000
- N Shares: 50
- SL, TL: 6%
- Buy/Sell: Close
- Borrow Rate: 0.25%

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ta

sns.set_theme()

In [2]:
data = pd.read_csv('aapl_5m_train.csv').dropna()

In [3]:
data.head()

,Unnamed: 0,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,Volume
0,0,1609770600,0,2021-01-04 14:30:00,133.570007,133.611602,132.389999,132.809997,6624663.0
1,1,1609770900,0,2021-01-04 14:35:00,132.750000,132.750000,131.809997,131.889999,2541553.0
2,2,1609771200,0,2021-01-04 14:40:00,131.500000,132.339996,131.500000,132.059997,2492415.0
3,3,1609771500,0,2021-01-04 14:45:00,132.000000,132.250000,131.899993,132.250000,1859131.0
4,4,1609771800,0,2021-01-04 14:50:00,132.000000,132.018096,131.520004,131.589996,1780105.0


In [4]:
rsi_indicator = ta.momentum.RSIIndicator(close=data.Close, window=15)

In [5]:
data['rsi'] = rsi_indicator.rsi()

In [6]:
historic = data.dropna()
historic['buy_signal'] = historic.rsi < 20
historic['sell_signal'] = historic.rsi > 80

/var/folders/60/rl4yk8jj3453bx7hbt074rbc0000gn/T/ipykernel_32980/3384102837.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historic['buy_signal'] = historic.rsi < 20
/var/folders/60/rl4yk8jj3453bx7hbt074rbc0000gn/T/ipykernel_32980/3384102837.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historic['sell_signal'] = historic.rsi > 80
